# Ejemplo sobre métricas de distancia

En este ejemplo vamos a realizar la importación de datos mediante pandas. Una vez importados los datos, vamos a computar las distancias existentes entre las diversas muestras importadas. De modo ilustrativo se emplearán cuatros métricas de distancias: la distancia basada en la cardinalidad del conjunto de intersección entre dos conjuntos dados; la métrica de Jaccard; la distancia del coseno y la distancia del coseno suavizada. 

In [1]:
import urllib.request
import os.path
import pandas as pd
import numpy as np
from numpy import zeros
import random as rd
from scipy.sparse import coo_matrix, csr_matrix
import time
from sklearn.preprocessing import normalize
from collections import defaultdict

Tras realizar la importación de los paquetes que emplearemos, cargamos los datos. Para ello hacemos uso de pandas. Los datos a emplear continenen información sobre el número de veces que un conjunto de usuarios han reproducido música de un cierto artista. 

In [2]:
if not os.path.isfile("prueba.tsv"):
    url = "https://www.dropbox.com/s/na8xphvfglsf2to/prueba.tsv?dl=1"
    u = urllib.request.urlopen(url)
    data = u.read()
    u.close() 
    with open("prueba.tsv", "wb") as f :
        f.write(data)

data = pd.read_table("prueba.tsv", usecols=[0, 2, 3], names=['user', 'artist', 'plays'])


In [3]:
data

,user,artist,plays
0,8375184063830301f3ef296da9f0a35a33186537,modena city ramblers,181
1,f11e905505496c62b894a10782c9db5fa6fb30cf,lauryn hill,33
2,f6083d24ae24c20727a8454555834e0cfd74e2fc,disturbed,4
3,b97522a230aca2c1693fe31847ebc93017e4daee,beyoncé,294
4,bc39901365f1e8ebf62aa6f5c10ba8a1ea4439da,elliott smith,73
...,...,...,...
999633,2fdc2ed71a16cfee4f3732f05f369905cdf2758e,broilers,47
999634,ae2ca0e73140292b4479e88b70f85d2bfc2021e3,pono,14
999635,cb91a9974e7fd3a0fd536abb12fd330a38dcba90,skunk anansie,17
999636,9bf5ba46bebefee77e5eb1f8eae9bf6d4159a62a,europe,10


De cara al análisis posterior, vamos a agrupar los datos creando los conjuntos necesarios. Primero agrupamos en base a cada uno de los artistas. Tras ello, codificamos tanto los usuarios como los artistas como una categoría de pandas. Creadas las categorías, extraemos los conjuntos de usuarios vinculados a cada artista. 

In [4]:
user_count = data.groupby('artist').size()
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
artist_sets = dict((artist, set(users)) for artist, users in data.groupby('artist')['user'])

In [5]:
data.head()

,user,artist,plays
0,8375184063830301f3ef296da9f0a35a33186537,modena city ramblers,181
1,f11e905505496c62b894a10782c9db5fa6fb30cf,lauryn hill,33
2,f6083d24ae24c20727a8454555834e0cfd74e2fc,disturbed,4
3,b97522a230aca2c1693fe31847ebc93017e4daee,beyoncé,294
4,bc39901365f1e8ebf62aa6f5c10ba8a1ea4439da,elliott smith,73


In [6]:
artists_ = dict(enumerate(data['artist'].cat.categories))   

También nos será útil tener una lista ordenada con los artistas. Para ello usamos la función sorted y le pasamos como segundo argumento la función en base a la cual vamos a ordenar la lista. Esa función se puede definir mediante el operador lambda. En este caso tendremos que para un usuario x el operador lambda devuelve el número de usuarios que han reproducido canciones de un artista. Este número es devuelto como un valor negativo, de modo que la lista se ordena del valor menor al mayor forzando que el primer elemento sea aquél que tiene el mayor número de reproducciones. 

In [7]:
to_generate = sorted(list(artists_),key=lambda x: -user_count[x])

La forma más directa para determinar cuán parecidos son dos conjuntos categoriales de datos es mediante el cómputo del número de elementos que tienen en común. 

In [8]:
#La forma más fácil y directa: compruebo si existen artitas comunes entre discos
def solapamiento(a, b):
    return len(a.intersection(b))

Para conseguir una medida más precisa sobre el parecido de dos conjuntos de datos categoriales, se puede normalizar el anterior resultado simplemente dividiéndolo por el número de elementos que están bien en el primer conjunto, bien en el segundo.

In [9]:
def jaccard(a,b):
    inter = float(len(a.intersection(b)))
    return inter/(len(a) +len(b) - inter)

In [10]:
start = time.time()
df = pd.DataFrame(columns=('artista1','artista2','solapamiento','Jaccard'))
i = 0
for artist in to_generate[1:2]:
    set1 = artist_sets[artists_[artist]]
    for art in to_generate[2:103]:
        if art != artist:
            set2 = artist_sets[artists_[art]]            
            df.loc[i] = [artists_[artist],artists_[art],solapamiento(set1,set2),jaccard(set1,set2)]   
            i += 1    
print(df)
loadEnd = time.time()
print("load time "+ str(loadEnd - start))

      artista1               artista2 solapamiento   Jaccard
0    radiohead               coldplay           90  0.011258
1    radiohead  red hot chili peppers           49  0.006909
2    radiohead                   muse           84  0.012045
3    radiohead             pink floyd           41  0.005983
4    radiohead              metallica           26  0.003787
..         ...                    ...          ...       ...
96   radiohead            norah jones           15  0.002828
97   radiohead                ramones           11  0.002073
98   radiohead             nickelback            8  0.001509
99   radiohead                the who           23  0.004355
100  radiohead         regina spektor           25  0.004742

[101 rows x 4 columns]
load time 0.18210625648498535


Otra medida que se suele emplear para medir la distancia entre datos categoriales es la medida del coseno. 

In [11]:
def cosine(a, b):
    return np.dot(a, b.T)[0, 0] / (norm2(a) * norm2(b))

def norm2(v):
    return np.sqrt((v.data ** 2).sum())

In [12]:
#asignamos un id a cada usuario. Para ello creamos un diccionario por defecto empleando el operador lambda. 
#Tras ello mapeamos un id a cada valor del diccionario, siendo el valor cada uno de los usuarios que tenemos. 
userids = defaultdict(lambda: len(userids))
data['userid'] = data['user'].map(userids.__getitem__)
artists = dict((artist, csr_matrix(
                (group['plays'], (zeros(len(group)), group['userid'])),
                shape=[1, len(userids)]))
        for artist, group in data.groupby('artist'))


In [13]:
data.groupby('artist').head()

,user,artist,plays,userid
0,8375184063830301f3ef296da9f0a35a33186537,modena city ramblers,181,172128
1,f11e905505496c62b894a10782c9db5fa6fb30cf,lauryn hill,33,316477
2,f6083d24ae24c20727a8454555834e0cfd74e2fc,disturbed,4,323037
3,b97522a230aca2c1693fe31847ebc93017e4daee,beyoncé,294,242853
4,bc39901365f1e8ebf62aa6f5c10ba8a1ea4439da,elliott smith,73,246544
...,...,...,...,...
999518,b29bf71adeba3b057c7ba3928d9e7403a09213e8,druhá tráva,67,233996
999530,1f10fd8665ee64c91b1885a00a8161e1e7d0b0da,daniele silvestri & subsonica,11,40795
999538,243c5029beea3320ee30e690d7a148fcc2399f04,lady alma,50,47416
999567,2d82d71cd03244c2c6fc381d811eb22c8f28b9ef,zatokrev,330,59612


In [14]:
for artist in to_generate[2:100]:
    print(cosine(artists['radiohead'],artists[artists_[artist]]))

0.00789347807501069
0.022318506370666746
0.004381875915778587
0.00395019381788708
0.0056511272988250495
0.007031559405047427
0.0014372226330307874
0.00256339273094042
0.0004902134348568248
0.00022102846299853996
0.004306918964438808
0.011935500860442838
0.005153591425129064
0.008061905554987061
0.012729293446214643
0.003248009450472991
0.0095503152607265
0.0037007284074806667
0.004747613212891607
0.005101176700385703
0.017860666258345186
0.006739938717700872
0.0024702255091381855
0.002552225689430623
0.009033834282938092
0.004452452620623447
0.008513732367246148
0.01379102860292077
0.005473861764375433
0.005660681043842465
0.009565450900397182
0.005174163734177935
0.0038199687113924796
0.0026915776727666372
0.00043475036670630516
0.0029686671569486528
0.0170963083977027
0.0022014646447658793
0.009725872340629382
0.0066909974385932205
0.0015518572038316525
0.0003520727088243165
0.004514966145619515
0.003463358367682004
0.017959152178670355
0.00026220479444719947
0.0001324410621821594
0.

In [15]:
SMOOTHING = 20

def smoothed_cosine(a, b):
    
    overlap = np.dot(binarize(a), binarize(b).T)[0, 0]

    
    return (overlap / (SMOOTHING + overlap)) * cosine(a, b)

def binarize(artist):
    ret = csr_matrix(artist)
    ret.data  = np.ones(len(artist.data))
    return ret

In [16]:
for artist in to_generate[2:100]:
    print(smoothed_cosine(artists['radiohead'],artists[artists_[artist]]))

0.006458300243190564
0.015849374089314066
0.003539207470436551
0.0026550483038257425
0.00319411542977068
0.004868002665032834
0.0007839396180167931
0.0018975764371896616
0.0002622071860862086
0.00011320970056022778
0.0030401780925450412
0.007819810908565997
0.0032088399439482848
0.005581319230375657
0.008262874693156874
0.0022002644664494454
0.006656280333233622
0.0024462442015550172
0.0031910187168615713
0.0032118519965391467
0.011701815824433052
0.004043963230620523
0.0012351127545690928
0.001589121655683218
0.006376824199721007
0.0027722818203881837
0.005301003172058922
0.009116103652778136
0.0031930860292190026
0.002681375231293799
0.007143817761056123
0.0035050786586366656
0.0024796288126582764
0.0017634474407781417
0.00022267701709347337
0.001169474940616136
0.011581370204895377
0.0010427990422575218
0.006822626865814641
0.004600060739032839
0.0008300631555378606
0.00014497111539824796
0.002842756462056732
0.0016872771534861045
0.010775491307202213
5.2440958889439896e-05
5.217375

## Primer ejercicio: 
Crear un dataframe de pandas para computar las distancias del coseno y el coseno suavizado de un artista dado a un conjunto de artistas. Realizad este ejercicio de acuerdo al procedimiento empleado en el caso de la distancias de solapamiento y la de Jaccard. 

Para este ejercicio se reutiliza el código anterior en dos funciones, una para generar el diccionario de conjuntos y otro para calcular las distancias en función de los artistas deseados.

In [17]:
def generateIds(data, set_name, set_elements):
    
    userids = defaultdict(lambda: len(userids))
    data['userid'] = data[set_elements].map(userids.__getitem__)
    artists = dict((artist, csr_matrix(
                    (group['plays'], (zeros(len(group)), group['userid'])),
                    shape=[1, len(userids)]))
            for artist, group in data.groupby(set_name))
    
    return artists

def computeCosineDistances(sets_dict, A, B):
    
    df = pd.DataFrame(columns=('artista1','artista2','cosine','smoothed_cosine'))
    
    
    i=0
    for artist1 in A:
        set1 = sets_dict[artist1]
        for artist2 in B:
            if artist1 != artist2:
                set2 = sets_dict[artist2]            
                df.loc[i] = [artist1, artist2, cosine(set1,set2), smoothed_cosine(set1,set2)]   
                i += 1 
            
    return df

In [18]:
sets_dictionary = generateIds(data=data, set_name='artist', set_elements='user')     
computeCosineDistances(sets_dict=sets_dictionary,
                       A=[' the ranting gryphon', 'radiohead'],
                       B=[' amy winehouse'])

,artista1,artista2,cosine,smoothed_cosine
0,the ranting gryphon,amy winehouse,0.0,0.0
1,radiohead,amy winehouse,0.0,0.0
